In [1]:
import re
import math
import warnings
import tqdm
import matplotlib
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from scipy.optimize import fsolve
from sympy import symbols, Eq, solve
from matplotlib.pyplot import MultipleLocator
warnings.filterwarnings("ignore")


%matplotlib inline
plt.rcParams['font.sans-serif'] = ['SimHei'] 
plt.rcParams['axes.unicode_minus'] = False 

/root/miniconda3/envs/jupyter-env/lib/python3.11/site-packages/scipy/__init__.py:155: UserWarning: A NumPy version >=1.18.5 and <1.26.0 is required for this version of SciPy (detected version 1.26.0
  warnings.warn(f"A NumPy version >={np_minversion} and <{np_maxversion}"


In [2]:
data1=pd.read_excel('/root/GDUT/q21/2a数据.xlsx')
data2=pd.read_excel('/root/GDUT/q21/时间点.xlsx')
data_f_1=pd.read_excel('/root/GDUT/附表1-检索表格-流水号vs时间.xlsx')
data_f_time=pd.read_excel('/root/GDUT/q21/时间点.xlsx')
data1=data1.rename(columns={'首次检查流水号':'入院首次影像检查流水号'})

In [3]:
# 转换为时间戳格式
columns_to_convert = data_f_time.columns[1:]

data_f_time[columns_to_convert] = data_f_time[columns_to_convert].apply(pd.to_datetime)
# Datetime对象转换为秒级的时间戳形式
data_f_time[columns_to_convert] = data_f_time[columns_to_convert].apply(lambda x: x.astype(int) // 10**9)
# 将负数的时间戳转变为空值
data_f_time[data_f_time.columns[1:]] = data_f_time[data_f_time.columns[1:]].apply(lambda x: x.mask(x < 0))


In [4]:
# 找到两个 DataFrame 共有的列名
common_columns = data_f_time.columns.intersection(data1.columns)

# 使用 data_f_time 中的列替换 _1a 中的相同列名的列

data1[common_columns] = data_f_time[common_columns]



In [5]:
Err=[data1['入院首次影像检查流水号'][i]-data1['发病到首次影像检查时间间隔'][i]*3600 for i in range(len(data1['发病到首次影像检查时间间隔']))]
for i in range(len(list(data1.columns))):
    if list(data1.columns)[i][-3:]=='流水号':
        for j in range(len(data1[list(data1.columns)[i]])):
            data1[list(data1.columns)[i]][j]-=Err[j]

In [6]:
# pd.set_option('display.max_rows',None)

# #设置列不限制数量
# pd.set_option('display.max_columns',None)

# #设置value的显示长度为100，默认为50
# pd.set_option('max_colwidth',100)
# data1.to_excel('2a数据_已替换时间戳_已矫正.xlsx', index=False)
data1.head()

,ID,发病到首次影像检查时间间隔,入院首次影像检查流水号,ED_volume.0,随访1流水号,ED_volume.1,随访2流水号,ED_volume.2,随访3流水号,ED_volume.3,随访4流水号,ED_volume.4,随访5流水号,ED_volume.5,随访6流水号,ED_volume.6,随访7流水号,ED_volume.7,随访8流水号,ED_volume.8
0,sub001,2.5,9000,48919,29766.0,57898.0,475590.0,81747.0,935055.0,107793.0,1531938.0,126558.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,sub002,3.0,10800,23526,53724.0,23390.0,249199.0,28415.0,1612859.0,34868.0,4029091.0,0.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,sub003,2.0,7200,32621,34281.0,41292.0,142545.0,49767.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,sub004,1.0,3600,22191,61107.0,18956.0,301843.0,9540.0,819004.0,5204.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,sub005,5.0,18000,47392,95283.0,23182.0,352595.0,12898.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [7]:
#对齐x0y0
# x0=data1['入院首次影像检查流水号']
# y0=data1['ED_volume.0']
# for i in range(len(list(data1.columns))):
#     if list(data1.columns)[i][-3:]=='流水号':
#         for j in range(len(data1[list(data1.columns)[i]])):
#             data1[list(data1.columns)[i]][j]-=x0[j]
#             data1[list(data1.columns)[i+1]][j]-=y0[j]
# from sklearn.svm import SVR
# from sklearn.metrics import r2_score
# # 绘制拟合结果
# import matplotlib.pyplot as plt
# import statsmodels.api as sm


x=[]
y=[]
ID=[]
datapp=pd.DataFrame()

# data1=data1[['ID']+list(data1.columns)[4:]]

# Err=[data1['入院首次影像检查流水号'][i]-data1['发病到首次影像检查时间间隔'][i]*3600 for i in range(len(data1['发病到首次影像检查时间间隔']))]
# for i in range(len(list(data1.columns))):
#     if list(data1.columns)[i][-3:]=='流水号':
#         for j in range(len(data1[list(data1.columns)[i]])):
#             data1[list(data1.columns)[i]][j]-=Err[j]

for i in range(len(list(data1.columns))):
    if list(data1.columns)[i][-3:]=='流水号':
        for j in range(len(data1[list(data1.columns)[i]])):
            if j <=99 and not math.isnan(data1[list(data1.columns)[i]][j]) and not math.isnan(data1[list(data1.columns)[i+1]][j]):
                x.append(data1[list(data1.columns)[i]][j]/3600)
                y.append( data1[list(data1.columns)[i+1]][j]/1000)
                ID.append(data1['ID'][j])
datapp['患者ID']=ID
datapp['影像时间']=x
datapp['水肿体积']=y
# datapp['Intercept'] = 1 
print(datapp)
# 使用线性混合效应模型建立纵向数据模型
# model = sm.MixedLM(datapp['水肿体积'], datapp[['Intercept','影像时间']],datapp['患者ID'])
# result = model.fit()

# # 查看模型结果
# print(result.summary())

# import statsmodels.api as sm

# # 假设你有一个名为data的pandas DataFrame，包含相关变量
# # 使用GEE模型拟合非线性二元模型
# model = sm.GEE.from_formula("水肿体积 ~ 影像时间", data=datapp, groups="患者ID",family=sm.families.Binomial(link=sm.families.links.Logit()))
# result = model.fit()

# # 查看模型结果摘要
# print(result.summary())

# plt.scatter(x,y)
# # plt.xlim([0,10])
# plt.show()
# from scipy.optimize import curve_fit

# # 多高斯函数定义
# def multi_gaussian(x, *params):
#     num_gaussians = len(params) // 3
#     gaussians = np.zeros_like(x)
#     for i in range(num_gaussians):
#         a = params[i*3]
#         b = params[i*3 + 1]
#         c = params[i*3 + 2]
#         gaussians += a * np.exp(-(x - b)**2 / (2 * c**2))
#     return gaussians

# # 初始参数猜测值
# initial_guess = [1, 2, 1, 3, 4, 1]

# # 使用curve_fit函数进行多高斯函数拟合
# popt, pcov = curve_fit(multi_gaussian, x, y, p0=initial_guess)

# # 绘制原始数据和拟合曲线
# plt.scatter(x, y, label='Data')
# plt.plot(x, multi_gaussian(x, *popt), 'r-', label='Fit')
# y1=multi_gaussian(x, *popt)
# import statsmodels.api as sm
# from sklearn.datasets import make_regression

# # 示例数据


# # 添加常数列作为截距
# x = sm.add_constant(x)

# # 拟合线性回归模型
# model = sm.OLS(y, x)
# results = model.fit()

# # 提取R2值和P值
# r2 = results.rsquared
# p_values = results.pvalues

# print("R2 score:", r2)
# print("P values:", p_values)


       患者ID         影像时间    水肿体积
0    sub001     2.500000  48.919
1    sub002     3.000000  23.526
2    sub003     2.000000  32.621
3    sub004     1.000000  22.191
4    sub005     5.000000  47.392
..      ...          ...     ...
445  sub060   294.407778  22.092
446  sub063   978.013333  47.223
447  sub068  4801.220556   0.000
448  sub056   457.617500   0.075
449  sub060   409.010556  20.720

[450 rows x 3 columns]


In [ ]:
from scipy import optimize




#线性
def liner_func(x,a,b,c,d):
    return a*x+b

#二次
def erchi_func(x,a,b,c,d):
    return a*x**2+b*x+c

#三次
def sanchi_func(x,a,b,c,d):
    return a*x**3+b*x**2+c*x+d

# 三角函数
def trig_func(x,a,b,c,d):
    return a*np.sin(x)+b*np.cos(x)+c
# 指数曲线
def target_func(x, a, b, c, d):
    return a * np.exp(-x / b) + c
# 对数函数
def hyp_func(x, a,b,c,d):
    return a*np.log(x)+b

# 定义高斯函数
def gaussian(x, a, b, c,d):
    return a * np.exp(-(x - b) ** 2 / (2 * c ** 2))

#定义Logistic函数
def logistic(x,a,b,c,d):
    return  a / (1 + exp(-b * (x - c)))


def __sst(y_no_fitting):
    """
    计算SST(total sum of squares) 总平方和
    :param y_no_predicted: List[int] or array[int] 待拟合的y
    :return: 总平方和SST
    """
    y_mean = sum(y_no_fitting) / len(y_no_fitting)
    s_list =[(y - y_mean)**2 for y in y_no_fitting]
    sst = sum(s_list)
    return sst


def __ssr(y_fitting, y_no_fitting):
    """
    计算SSR(regression sum of squares) 回归平方和
    :param y_fitting: List[int] or array[int]  拟合好的y值
    :param y_no_fitting: List[int] or array[int] 待拟合y值
    :return: 回归平方和SSR
    """
    y_mean = sum(y_no_fitting) / len(y_no_fitting)
    s_list =[(y - y_mean)**2 for y in y_fitting]
    ssr = sum(s_list)
    return ssr


def __sse(y_fitting, y_no_fitting):
    """
    计算SSE(error sum of squares) 残差平方和
    :param y_fitting: List[int] or array[int] 拟合好的y值
    :param y_no_fitting: List[int] or array[int] 待拟合y值
    :return: 残差平方和SSE
    """
    s_list = [(y_fitting[i] - y_no_fitting[i])**2 for i in range(len(y_fitting))]
    sse = sum(s_list)
    return sse


def goodness_of_fit(y_fitting, y_no_fitting):
    """
    计算拟合优度R^2
    :param y_fitting: List[int] or array[int] 拟合好的y值
    :param y_no_fitting: List[int] or array[int] 待拟合y值
    :return: 拟合优度R^2
    """
    SSR = __ssr(y_fitting, y_no_fitting)
    SST = __sst(y_no_fitting)
    rr = SSR /SST
    return rr
def selected_func(you,model,model_select):
    maxyou=max(you)
    for s in range(len(you)):
        if you[s]==maxyou:
            
            return model[s],model_select[s]
        
    

def is_non_real(x):
    # 判断复数是否为非实数
    return math.isinf(x) or math.isnan(x)

def sgn(x):
    if x < 0:
        return -1
    elif x == 0:
        return 0
    else:
        return 1
        
#========================粒子群=======================================
import numpy as np



# def particle_swarm_optimization(mm, objective_func, num_particles, max_iterations,parm):
#     # 初始化参数
#     dimensions = 1
#     inertia = 0.5  # 惯性权重
#     cognitive_weight = 0.2  # 学习因子
#     social_weight = 0.2  # 学习因子
#     min_bound = 0  # 变量的最小边界
#     max_bound = 48  # 变量的最大边界

#     # 初始化粒子的位置和速度
#     particles = np.random.uniform(min_bound, max_bound, (num_particles, dimensions))
#     velocities = np.zeros((num_particles, dimensions))
#     if mm==1:
#         # 初始化粒子的局部最佳位置和全局最佳位置
#         personal_best_positions = particles.copy()
#         global_best_position = particles[np.argmin(objective_func(particles,parm[0],parm[1],parm[2],parm[3]))]

#         # 迭代更新粒子的速度和位置
#         for _ in range(max_iterations):
#             for i in range(num_particles):
#                 # 更新粒子的速度
#                 velocities[i] = (inertia * velocities[i] +
#                                 cognitive_weight * np.random.rand() * (personal_best_positions[i] - particles[i]) +
#                                 social_weight * np.random.rand() * (global_best_position - particles[i]))

#                 # 限制速度范围
#                 velocities[i] = np.clip(velocities[i], min_bound, max_bound)

#                 # 更新粒子的位置
#                 particles[i] += velocities[i]

#                 # 限制位置范围
#                 particles[i] = np.clip(particles[i], min_bound, max_bound)

#                 # 更新局部最佳位置和全局最佳位置
#                 if objective_func(particles[i],parm[0],parm[1],parm[2],parm[3]) < objective_func(personal_best_positions[i],parm[0],parm[1],parm[2],parm[3]):
#                     personal_best_positions[i] = particles[i]

#                 if objective_func(particles[i],parm[0],parm[1],parm[2],parm[3]) < objective_func(global_best_position,parm[0],parm[1],parm[2],parm[3]):
#                     global_best_position = particles[i]
#         return global_best_position, objective_func(global_best_position,parm[0],parm[1],parm[2],parm[3])
#     if mm==-1:
#         # 初始化粒子的局部最佳位置和全局最佳位置
#         personal_best_positions = particles.copy()
#         global_best_position = particles[np.argmin(-objective_func(particles,parm[0],parm[1],parm[2],parm[3]))]

#         # 迭代更新粒子的速度和位置
#         for _ in range(max_iterations):
#             for i in range(num_particles):
#                 # 更新粒子的速度
#                 velocities[i] = (inertia * velocities[i] +
#                                 cognitive_weight * np.random.rand() * (personal_best_positions[i] - particles[i]) +
#                                 social_weight * np.random.rand() * (global_best_position - particles[i]))

#                 # 限制速度范围
#                 velocities[i] = np.clip(velocities[i], min_bound, max_bound)

#                 # 更新粒子的位置
#                 particles[i] += velocities[i]

#                 # 限制位置范围
#                 particles[i] = np.clip(particles[i], min_bound, max_bound)

#                 # 更新局部最佳位置和全局最佳位置
#                 if -objective_func(particles[i],parm[0],parm[1],parm[2],parm[3]) < -objective_func(personal_best_positions[i],parm[0],parm[1],parm[2],parm[3]):
#                     personal_best_positions[i] = particles[i]

#                 if -objective_func(particles[i],parm[0],parm[1],parm[2],parm[3]) < -objective_func(global_best_position,parm[0],parm[1],parm[2],parm[3]):
#                     global_best_position = particles[i]
#         # 输出结果
#         return global_best_position, -objective_func(global_best_position,parm[0],parm[1],parm[2],parm[3])

# 求解方程 f(x) = y
def solve_equation(f,y):
    equation = lambda x: f(x) - y
    x_initial_guess = 0  # 初始猜测值
    x_solution = fsolve(equation, x_initial_guess)
    return x_solution

data=datapp    #data->['患者ID','影像时间','水肿体积']        
x=data['影像时间']
y=data['水肿体积']

Set={}
youSet={}
resSet={}
resTime={}
# for index, row in selected_df.iterrows():
#     if index >= 100:
#         break
#     # 去除NaN值
    # data = row.values[~np.isnan(row.values)]
    # # 将数据分成x和y坐标对
    # x = data[::2] / (3600 )
    # y = data[1::2]/1000
you=[]
model=[]
model_select=[]
you_index={}
try:
    a1 = np.polyfit(x, y, 1)#线性
    you1 = goodness_of_fit([liner_func(x[p],a1[0],a1[1],0,0) for p in range(len(x))],y)
    you.append(you1)
    model.append({'回归类型':'线性回归','回归系数':a1})
    you_index['线性回归']=you1
    model_select.append((liner_func,a1+[0,0]))
except:
    pass

try:
    a2 = np.polyfit(x, y, 2)#二次
    you2 = goodness_of_fit([erchi_func(x[p],a2[0],a2[1],a2[2],0) for p in range(len(x))],y)
    you.append(you2)
    model.append({'回归类型':'二次函数回归','回归系数':a2})
    you_index['二次函数回归']=you2
    model_select.append((erchi_func,a2+[0]))
except:
    pass

try:
    a3 = np.polyfit(x, y, 3)#三次
    you3 = goodness_of_fit([sanchi_func(x[p],a3[0],a3[1],a3[2],a3[3]) for p in range(len(x))],y)
    you.append(you3)
    model.append({'回归类型':'三次函数回归','回归系数':a3})
    you_index['三次函数回归']=you3
    model_select.append((sanchi_func,a3))
except:
    pass

#拟合三角函数模型
try:
    a4,_=optimize.curve_fit(trig_func,x,y)
    you4 = goodness_of_fit([trig_func(x[p],a4[0],a4[1],a4[2],0) for p in range(len(x))],y)
    you.append(you4)
    model.append({'回归类型':'三角函数回归','回归系数':a4})
    you_index['三角函数回归']=you4
    model_select.append((trig_func,a4+[0]))

except:
    pass

#拟合指数函数模型
try:
    a5,_=optimize.curve_fit(target_func,x,y)
    you5 = goodness_of_fit([target_func(x[p], a5[0], a5[1], a5[2],0,0) for p in range(len(x))],y)
    you.append(you5)
    model.append({'回归类型':'指数函数回归','回归系数':a5})
    you_index['指数函数回归']=you5
    model_select.append((target_func,a5+[0]))
except:
    pass

#拟合对数函数模型
try:
    a6,_=optimize.curve_fit(hyp_func,x,y)
    you6 = goodness_of_fit([hyp_func(x[p], a6[0],a6[1],0,0) for p in range(len(x))],y)
    you.append(you6)
    model.append({'回归类型':'对数函数回归','回归系数':a6})
    you_index['对数函数回归']=you6
    model_select.append((hyp_func,a6+[0,0]))
except:
    pass

popt, pcov = curve_fit(gaussian, x, y)
# 提取拟合参数
a_fit, b_fit, c_fit = popt
you7 = goodness_of_fit([gaussian(x[p], a_fit,b_fit,c_fit,0) for p in range(len(x))],y)
you.append(you7)
model.append({'回归类型':'高斯函数回归','回归系数':popt})
you_index['高斯函数回归']=you7
model_select.append((hyp_func,popt+[0]))


try:
    popt, pcov = curve_fit(logistic, x, y)
    # 提取拟合参数
    a_fit, b_fit, c_fit = popt
    you8 = goodness_of_fit([logistic(x[p], a_fit,b_fit,c_fit,0) for p in range(len(x))],y)
    you.append(you8)
    model.append({'回归类型':'logistic函数回归','回归系数':popt})
    you_index['logistic函数回归']=you8
    model_select.append((hyp_func,popt+[0]))
except:
    pass

# model=[f'线性回归，系数为{a1}',f'二次函数回归，系数为{a2}',f'三次函数回归，系数为{a3}',f'三角函数回归，系数为{a4}',f'指数函数回归，系数为{a5}',f'对数函数回归，系数为{a6}']
res,model_=selected_func(you,model,model_select)

# Set[index]=res#选择函数模型，例如：1用三角函数
# youSet[index]=you_index#拟合优度
print(model)
print(you_index)
#     best_position1, best_value1 = particle_swarm_optimization(1,model_[0], 50, 500,model_[1])
#     # best_value1=round(float(best_value1),4)
#     # print("最小值:",best_value1,'最小位置:',best_position1)
#     best_position2, best_value2 = particle_swarm_optimization(-1,model_[0], 50, 500,model_[1])

#     # best_value2=round(float(-best_value2),4)
#     # print("最大值:", -best_value2,'最大位置:',best_position2)
#     resSet[index+1]=(y[0],float(-best_value2),x[0],best_position2)
#     x1 = symbols('x')
#     def y1(x):
#         return model_[0](x,*model_[1])
#     if float(best_value1)<0:
#         best_value1=0.1
    
#     # 在第一次检测的的右边的非复数解
    
#     solution11 = solve_equation(y1,float(y[0])+6)
#     solution1=[]
#     for u in solution11:
#         if str(u)[-1]!='I' and u>=x[0]:
#             solution1.append(u)
       
#     solution22 = solve_equation(y1,float(y[0])*1.33)
#     solution2=[]
#     for u in solution22:
#         if str(u)[-1]!='I' and u>=x[0]:
#             solution2.append(u)

#     if solution1 == [] and solution2 == []:
#         resTime[index+1]=-999
#     elif solution1 == [] and solution2 != []:
#         if solution2[0] <= 48:
#             resTime[index+1]=round(solution2[0],4)
#         else:
#             resTime[index+1]=-999
#     elif solution2 == [] and solution1 != []:
#         if solution1[0] <= 48:
#             resTime[index+1]=round(solution1[0],4)
#         else:
#             resTime[index+1]=-999
#     else:
#         if solution1[0] > 48 and solution2[0] > 48:
#             resTime[index+1]=-999
#         else:
#             resTime[index+1]=round(min(solution1[0],solution2[0]),4)
#             +[0]+[0]
    

# dicts1 = [{key: value for key, value in Set[row].items()} for row in Set]
# df1 = pd.DataFrame(dicts1)
# df1.to_excel('血肿1a_100人拟合函数选择结果.xlsx')
# dicts2 = [{key: value for key, value in youSet[row].items()} for row in youSet]
# df2 = pd.DataFrame(dicts2)
# df2.to_excel('血肿1a_100人拟合优度.xlsx')

# # kuozan_index={}
# # for i in resSet:
# #     kuozan_index[i]=[0,1][sgn(resSet[i][3]-resSet[i][2])*(resSet[i][1]-resSet[i][0])>=6 or resSet[i][1]>=1.33*resSet[i][0]]
# # df111=pd.DataFrame(columns=['ID','是否扩张'])
# # df111['ID']=kuozan_index.keys()
# # df111['是否扩张']=kuozan_index.values()
# # df111.to_excel('是否扩张.xlsx',index=False)


# Time=pd.DataFrame(columns=['ID','扩张时间'])
# Time['ID']=resTime.keys()
# Time['扩张时间']=resTime.values()
# Time.to_excel('扩张时间.xlsx',index=False)